In [33]:
# %%capture

import sys
import os
import importlib
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

import import_default
import import_database
import import_other

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)

from import_default import *
from import_database import *
from import_other import *

In [34]:
today_index_df = get_mongo_collection(stock_db, "today_index", find_query={'ticker': 'VNINDEX'}, projection={'_id': 0, 'date': 1, 'close': 1})
history_index_df = get_mongo_collection(stock_db, "history_index", find_query={'ticker': 'VNINDEX'}, projection={'_id': 0, 'date': 1, 'close': 1})
vnindex_df = pd.concat([history_index_df, today_index_df], ignore_index=True).rename(columns={'close': 'vnindex'})

In [35]:
market_benchmark = get_mongo_collection(stock_db, "market_benchmark")
market_phase = get_mongo_collection(stock_db, "market_phase")
market_phase = market_phase.merge(vnindex_df, on='date', how='left')
market_phase['phase_name'] = market_phase['final_signal_phase'].apply(lambda x: 
                                    'An toàn' if x == 3 else 
                                    'Nghi ngờ (1)' if x == 2 else 
                                    'Nghi ngờ (2)' if x == 1 else 
                                    'Rủi ro')

In [37]:
%%capture
save_to_mssql(cts_engine, market_phase, 'vip_market_phase')
save_to_mssql(cts_engine, market_benchmark, 'vip_market_benchmark')